*  This program compares the performance 
*                             of the 4 investement rules/strategies 
*  when applies to the 25 FF portfolios                                                   

In [1]:
import pandas as pd                     # To load data, we use the package pandas
import numpy as np
import scipy.special as sc                # to compute the beta function 


# Load the data 

# Load the data 

df = pd.read_excel('Factors_July26_July11.xlsx')        # It has 5 columns:  date, mkt, size, b/m, riskree rate
                                                    # downloaded from Ken French's website
df2 = pd.read_excel('FF25_July26_July11.xlsx')       # return on 25 FF portfolios from Ken French's web

mkt = df.loc[:,"mkt"]/100                       # Mkt excess return;  divided by 100 due to data are in %
rf = df.loc[:,"rate"]/100

R1 = df2.loc[:,'FF1' : 'FF25']/100        # extract the 25 FF returns, R1 is T by 25. 

R1 = np.array(R1)                         # convert list to array to apply np.functions
rf = np.array(rf)                         # convert list to array to apply np.functions
T = len(df)                               # The number of obs
Re = np.ones((T,25))                           # creat storage for excess returns

for i in range(25):
    Re[:,i] = R1[:,i] - rf                # the excess return:  each indu substracts riskfree rate, Re[:,i]-rf

T0 = 120                 # set aside data for the initial estimation

print(' Total sample size \n')
print(T)
print(' Sample size used to start estimation \n')
print(T0)

 Total sample size 

1021
 Sample size used to start estimation 

120


In [2]:
# Compute the first three portfolio weights, which are simpler


TT = T - T0              #  from T0, T0+1, ..., to T-1
                         #  No need for T as we will not know its return

                         # declare storage for the 4 portfolios (though not necessary, but clearer)
                             #          each is N-vector over time
N=25
w1 = np.ones((N,TT))
w2 = np.ones((N,TT))
w3 = np.ones((N,TT))
w4 = np.ones((N,TT))

                    # 1):  1/N rule over T0, T0+1, ..., T-1
    
w1 = np.ones((N,TT)) / N       

                    # 2):  The other 3 rules

gamma = 3     
ones5 = np.ones((N,1))            # a 5-vector of ones, to be used later

for t in range(T0,T):
    RR = np.ones((t,N))           # to store the data from 1 to t, t by M matrix 
    for i in range (t):           # get available data up to T0
        RR[i,:] = Re[i,:] 
    mu = np.mean(RR, axis = 0)          # the mean taking each column of the matrix 
    mu = mu.T                           # make it a column vector
    V = np.cov(RR.T)                        # the covariance estimate 
    VI = np.linalg.inv(V)                     #  Inverse of V
    tt = t-T0
    w2[:,tt] = (1/gamma)*np.matmul(VI, mu)       # store the plug-in weights
    A = np.matmul(VI, ones5)
    b = np.matmul(ones5.T, A)
    w3C = A / b                                  #  N by 1, GMV weights
    for j in range(N):
        w3[j,tt] = w3C[j]                      # store to w3
                                  # Now the last and complex weights 
    B = np.matmul(VI, mu) 
    theta2 = np.matmul(mu.T, B)             # sample Sharpe ratio squared
    x = theta2 / (1+ theta2)                # get parameters for incomplete beta function
    a = N/2.0
    b = (t-N) / 2.0
    ibeta = sc.betainc(a, b,x)*sc.beta(a, b)   # b/c sc.betainc defined with 1/sc.beta(a, b) coeff
    c = ((t-N-2)*theta2 - N)/t
    d = 2*np.power(theta2,N/2.0)*np.power(1+theta2,-(t-2)/2.0)
    theta2a = c + d / (t*ibeta)
    c1 = (t-2)*(t-N-2) / ( (t-N-1)*(t-N-4) )
    A = np.matmul(V, ones5)                 # variables in pi1 and pi2
    B = np.matmul(ones5.T, A) 
    C = np.matmul(ones5.T, mu) / N
    pi1 = B / (N*N) - (2*C)/gamma + theta2a / (gamma*gamma)
    pi2 = (c1-1)*theta2a / (gamma*gamma) + c1*N / (t*gamma*gamma)
    if pi1 < 0:                            # sets pi1 to be 0 or positive
        pi1 = 0
    delta = pi1 / (pi1+pi2)
    w2C=((t-N-2)/t)*w2[:,tt]                # the adjusted plug-in rule
    w2C.shape = (N,1)                       # make sure the next Equ gets N by 1 vector
    w4C = (1-delta)*ones5/N +  delta*w2C     # the combination weights 
                    # (t-N-2)/t is due to using the adjusted plug-in
    for j in range(N):
        w4[j,tt] = w4C[j]                      # store to w4

print('The first weights at T0 \n')
print(w1[:,0])
print('The weights \n')
print(w2[:,0])
print('The weights \n')
print(w3[:,0])
print('The weights \n')
print(w4[:,0])

The first weights at T0 

[0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04
 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04 0.04]
The weights 

[-0.07983265 -0.49429106 -0.20456332 -0.36829845  1.78607477 -1.3925989
  0.42404006  0.32606216  0.62521021 -0.18152336 -1.24373889 -0.57073966
  3.24093094  0.02677053 -0.5297965   0.18641306  0.02745378 -0.94493392
  2.44011882 -0.97925924  0.34000889  2.80386517 -3.11878599 -0.80556802
 -0.45568581]
The weights 

[-0.10260343 -0.16837576 -0.18508745 -0.20168231  0.03170558  0.1372096
 -0.1650479   0.4639969   0.40558682  0.25568113  0.02868637  0.05523367
 -0.32569687  0.12485927 -0.34035639 -0.40158244  0.09770036  0.1486635
  0.28002868 -0.45945838  0.50109418  0.78283133  0.32038581 -0.25592381
 -0.02784847]
The weights 

[-0.00345363 -0.14046641 -0.04468743 -0.09881543  0.6133831  -0.43743145
  0.16311798  0.13072818  0.22962135 -0.03707082 -0.3882209  -0.16573899
  1.09433339  0.03178754 -0.15220389  0.0845626  

In [3]:
# Compute the Sharpe Ratios and Accu Return of the 4 rules and Mkt
        
       # write a function to do it to save space or to be cleaer

def get_SRW(w):
    Port=np.ones((TT,1))      #   to store the returns on the portfolio
    ExPort = np.ones((TT,1))      # to store excess return
    for t in range(TT):
        w5 = w[:,t]
        ExPort[t] = np.dot(w5,Re[T0+t])      # excess return of the portfolio
        Port[t] = ExPort[t] + rf[T0+t]       # add back r_f to get the raw return
    muP = ExPort.mean()
    sig2P = ExPort.var()
    sigP = np.sqrt(sig2P)
    SharpeP = np.sqrt(12)*muP/sigP        # annualize the Sharp Ratio
    CC_Port = np.ones((TT,1))          # to store the accumulative returns 
    CC_Port[0] = 1+Port[0]  
    for t in range(TT-1):
        CC_Port[t+1] = CC_Port[t]*(1+Port[t+1])
    c = CC_Port[TT-1]
    return SharpeP, c

sr1, c1 = get_SRW(w1)
sr2, c2 = get_SRW(w2)
sr3, c3 = get_SRW(w3)
sr4, c4 = get_SRW(w4)

# For the mkt
                                     # Sharpe ratio of the Market 
                                     #  over  T0+1, ..., T 
mkt1 = df.loc[T0:T,"mkt"]/100       # Note r_{T0+1} is stored as r[T0]
mu = mkt1.mean()                      
sig2 = mkt1.var()                       
sigma = np.sqrt(sig2)                  
SharpeM = np.sqrt(12)*mu/sigma      # annualized      

# Get the accu return for the mkt

CC = np.ones((TT,1))          # to store the accumulative returns 
                           # raw return by adding rf back
mkt2 = (df.loc[T0:T,"mkt"] + df.loc[T0:T,"rate"])/100  

print('The Mkt raw returns \n')
print (mkt2.head()) 

mkt2 = np.array(mkt2)       # the index will start from 0

CC[0] = 1 + mkt2[0]                 # initial accu return

for t in range(TT-1):
    CC[t+1] = CC[t]*(1+mkt2[t+1])
    
print('The Annualized Sharpe ratios of the Opt Port and Mkt \n')
print('          {0:.4f}   {1:.4f}  \n'.format(sr1, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr2, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr3, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr4, SharpeM)) 

print('Terminal wealth of the 4 rules  \n')
print(c1)  
print(c2)  
print(c3)  
print(c4) 
print('Terminal wealth in for investing in Mkt  \n')
print(CC[TT-1])  


The Mkt raw returns 

120    0.0646
121    0.0108
122    0.0143
123    0.0691
124    0.0333
dtype: float64
The Annualized Sharpe ratios of the Opt Port and Mkt 

          0.5214   0.4610  

          0.7748   0.4610  

          0.4899   0.4610  

          0.8408   0.4610  

Terminal wealth of the 4 rules  

[7848.276892]
[71192810.41209817]
[1626.85313738]
[5086908.95335802]
Terminal wealth in for investing in Mkt  

[1565.54242462]
